# Import Python Modules

Import the Python modules needed for this notebook.

In [ ]:
# The Google Earth Engine module
import ee
# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap
# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas
# The datetime module is used to specify the dates
# to search for imagery
import datetime
# The colab module to access data from your google drive
from google.colab import drive

## Install pb_gee_tools

The pb_gee_tools module (https://github.com/remotesensinginfo/pb_gee_tools) is a collection of tools used to make it easier to access datasets on Google Earth Engine.

The code below tests whether the module is available to import and if not will install the module from code repository.

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.utils
except:
  !git --version
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip --version
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.utils

# Autheticate with Google Earth Engine

The next stage is to specify the project being used for your analysis and to autheticate with Google Earth Engine.

The first time you run the following cell it will come up with a prompt asking your permission to access your GEE account and project.

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-pb-dev') # <==== Replace this with your own EE project string

# Mount Google Drive

To access files within your Google Drive we need to 'mount it' to this colab environment.

Again, it will come up with a prompt asking your permission.

In [ ]:
drive.mount('/content/drive')

# Customise Inputs

This next section is where you need to specify input variables to control the MODIS Albedo which is read.

You need to specify:

*   The path (on your google drive) to the vector layer specifying your region of interest. The vector layer can be any format that Geopandas is able to read (e.g., Shapefile, GeoJSON, GPKG).
*   The start date (i.e., earlier date) for the image search
*   The end date (i.e., later date) for the image search
*   The output no date value used - should always be 0
*   The output image file name (Note, do not specify the file extention - it will have .tif added when created)
*   The output directory on your google drive where the output image will be saved.



In [ ]:
vec_file = '/content/drive/MyDrive/test_shp/test_roi_poly.shp'
start_year = 2010
start_month = 1
n_months = 11 # In addition to start month so this will output a total of 12 months.
out_no_data_val = 0.0
modis_band_name = "Albedo_BSA_vis" # This is the band which will be outputted.
out_file_name = "modis_albedo_month_2010"
out_gdrive_dir = "test_gee_outs"

# Processing Stages

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
# Get the list of years and months
month_years = pb_gee_tools.utils.create_year_month_n_months_lst(start_year, start_month, n_months)

# Iterate through the months
img_lst = list()
for year, month in month_years:
  print(f"{month} / {year}")
  # Find the date of the last day of the month.
  end_day = pb_gee_tools.utils.find_month_end_date(year, month)

  # Specify the start and end years to search by
  start_date = datetime.datetime(year=year, month=month, day=1)
  end_date = datetime.datetime(year=year, month=month, day=end_day)

  # Find the MODIS albedo scenes
  modis_albedo_col = pb_gee_tools.datasets.get_modis_albedo_collection(
      aoi = tile_aoi,
      start_date = start_date,
      end_date = end_date,
  )

  # Check how many scenes there are in the returned image collection
  n_imgs = modis_albedo_col.size().toInt().getInfo()
  print(f"\tNumber of images: {n_imgs}")

  if n_imgs > 0:
    # If there are more than 0 scenes then calculate the median
    # and select the band which is to be outputted and rename it
    # to include the month/year.
    modis_albedo_med_img = modis_albedo_col.select([modis_band_name], [f"{modis_band_name}_{year}_{month}"]).median()

    # Add median image to output list.
    img_lst.append(modis_albedo_med_img)
  else:
    print(f"\tNo images found for {month} / {year}")

# Create an image collection from the list of monthly medians.
monthly_img_col = ee.ImageCollection.fromImages(ee.List(img_lst))

In [ ]:

# Export the median image to your Google Drive
task = ee.batch.Export.image.toDrive(
        image=monthly_img_col.toBands().int16(),
        description=out_file_name,
        folder=out_gdrive_dir,
        region=tile_aoi,
        scale=500,
        fileFormat="GeoTIFF",
        formatOptions={"cloudOptimized": True, 'noData': out_no_data_val},
)
task.start()